#### Day 6: The HAVING Clause 🎉

Today we explore the HAVING clause, which is used to filter results after they have been grouped.

#### 1. Database Connection

In [1]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="30_Days_SQL"
)
cursor = conn.cursor()
print("Connected to '30_Days_SQL'!")

Connected to '30_Days_SQL'!


---

#### Practice Exercises 📖

#### 1. Customers whose total order value exceeds $5000

In [2]:
extra_orders = [
    (107, 1, 1, '2024-01-10', 5200.00, 'High value order'),
    (108, 4, 4, '2024-03-20', 5500.00, 'Satisfied customer')
]
cursor.executemany("INSERT IGNORE INTO orders VALUES (%s, %s, %s, %s, %s, %s)", extra_orders)
conn.commit()

query = """
SELECT customer_id, SUM(order_total) AS total_value 
FROM orders 
GROUP BY customer_id 
HAVING total_value > 5000
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13300\2402562759.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,customer_id,total_value
0,1,10400.0
1,4,5500.0


#### 2. Products where the average price is greater than $50

In [3]:
query = """
SELECT category, AVG(price) AS average_price 
FROM products 
GROUP BY category 
HAVING average_price > 50
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13300\1539709902.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,category,average_price
0,Electronics,423.333333
1,Furniture,225.000000


#### 3. Authors with more than 3 published books and an average rating above 4.5

In [4]:
cursor.execute("DROP TABLE IF EXISTS books")
cursor.execute("DROP TABLE IF EXISTS authors")

cursor.execute('''
CREATE TABLE authors (
    author_id INT PRIMARY KEY,
    author_name VARCHAR(50)
)
''')

cursor.execute('''
CREATE TABLE books (
    book_id INT PRIMARY KEY,
    author_id INT,
    rating DECIMAL(2,1),
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
)
''')

authors_data = [(1, 'Author A'), (2, 'Author B')]
cursor.executemany("INSERT INTO authors VALUES (%s, %s)", authors_data)

books_data = [
    (1, 1, 4.6), (2, 1, 4.8), (3, 1, 4.7), (4, 1, 4.9),
    (5, 2, 4.2), (6, 2, 4.3), (7, 2, 4.1), (8, 2, 4.4)
]
cursor.executemany("INSERT INTO books VALUES (%s, %s, %s)", books_data)
conn.commit()

query = """
SELECT author_name, COUNT(*) AS book_count, AVG(rating) AS average_rating 
FROM authors a
JOIN books b ON a.author_id = b.author_id
GROUP BY author_name 
HAVING book_count > 3 AND average_rating > 4.5
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13300\2608945975.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,author_name,book_count,average_rating
0,Author A,4,4.75


#### 4. Departments where the total salary exceeds $100,000

In [5]:
query = """
SELECT department_id, SUM(salary) AS total_salary 
FROM employees 
GROUP BY department_id
HAVING total_salary > 100000
"""
pd.read_sql(query, conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13300\1479630755.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,department_id,total_salary
0,1,110000.0
1,3,175000.0


In [6]:
conn.close()